In [2]:
!pip install pyspark

     |████████████████████████████████| 204.2MB 62kB/s 
     |████████████████████████████████| 204kB 44.7MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=d04c30442aff11157006fbb834161dba314aaaccd9bd1ebd5b638972f1c591f4
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [13]:
!wget https://raw.githubusercontent.com/PeterZaidel/Mexmat/master/students_csv.csv

--2020-12-05 09:10:15--  https://raw.githubusercontent.com/PeterZaidel/Mexmat/master/students_csv.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 72036 (70K) [text/plain]
Saving to: ‘students_csv.csv’

students_csv.csv    100%[===================>]  70.35K  --.-KB/s    in 0.01s   

2020-12-05 09:10:15 (4.95 MB/s) - ‘students_csv.csv’ saved [72036/72036]



In [18]:
!ls 

sample_data  students_csv.csv


In [1]:
import pyspark
import pandas as pd
sc = pyspark.SparkContext(appName="PySparkIntro")

In [46]:
df = pd.read_csv("students_csv.csv")

In [47]:
df

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75
...,...,...,...,...,...,...,...,...
995,female,group E,master's degree,standard,completed,88,99,95
996,male,group C,high school,free/reduced,none,62,55,55
997,female,group C,high school,free/reduced,completed,59,71,65
998,female,group D,some college,standard,completed,68,78,77


In [ ]:
types = [
  'gender',
  'race/ethnicity',
  'parental level of education',
  'lunch',
  'test preparation course',
  'math score',
  'reading score',
  'writing score'
 ]
int_types = [
  'math score',
  'reading score',
  'writing score'       
 ]

def parse_row(s):
  args = s.replace('"', '').split(',')
  res = dict(zip(types, args))
  for k in int_types:
    res[k] = int(res[k])
  return res

In [33]:
data_rdd = sc.textFile("students_csv.csv")
header = data_rdd.first()
data_rdd = data_rdd.filter(lambda x: x != header).map(parse_row)
data_rdd.take(2)

[{'gender': 'female',
  'lunch': 'standard',
  'math score': 72,
  'parental level of education': "bachelor's degree",
  'race/ethnicity': 'group B',
  'reading score': 72,
  'test preparation course': 'none',
  'writing score': 74},
 {'gender': 'female',
  'lunch': 'standard',
  'math score': 69,
  'parental level of education': 'some college',
  'race/ethnicity': 'group C',
  'reading score': 90,
  'test preparation course': 'completed',
  'writing score': 88}]

Кто умнее - мальчики или девочки?

In [86]:
# MAP -> (key = gender, value = (mean score, 1))
smartest_gender_rdd = data_rdd.map(
    lambda x: (x['gender'], ((x['reading score'] + x['writing score'] + x['math score']) / 3.0 , 1))
)
print("After map: ", smartest_gender_rdd.take(1))
# REDUCE -> (gender, (sum scores, count))
smartest_gender_rdd = smartest_gender_rdd.reduceByKey(lambda a,b: (a[0] + b[0], a[1] + b[1]))
print("After reduce: ", smartest_gender_rdd.collect())
# CALCULATE MEAN SCORE BY GENDER
smartest_gender_rdd = smartest_gender_rdd.map(lambda x: (x[0], x[1][0]/x[1][1]))
smartest_gender_rdd = smartest_gender_rdd.sortBy(keyfunc=lambda x: x[1], ascending=False)

After map:  [('female', (72.66666666666667, 1))]
After reduce:  [('female', (36036.99999999999, 518)), ('male', (31733.66666666667, 482))]


In [80]:
smartest_gender_rdd.collect()

[('female', 69.56949806949805), ('male', 65.83748271092671)]

In [87]:
df.groupby('gender').gender.count()

gender
female    518
male      482
Name: gender, dtype: int64

Задание - посчитать средний math_score в зависимости от race/ethnicity и parental level of education

